Import library

In [ ]:
# ===============================
# CORE LIBRARIES
# ===============================
import numpy as np
import pandas as pd

# ===============================
# PREPROCESSING
# ===============================
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold

# ===============================
# BALANCING
# ===============================
from imblearn.over_sampling import SMOTE

# ===============================
# DEEP LEARNING
# ===============================
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

# ===============================
# EVALUATION
# ===============================
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ===============================
# REPRODUCIBILITY
# ===============================
np.random.seed(42)
tf.random.set_seed(42)

# ===============================
# LOAD DATASET
# ===============================
data_path = "/content/drive/MyDrive/JST_TB/Dataset1/csv_result-chronic_kidney_disease.csv"
data = pd.read_csv(data_path, on_bad_lines='skip')

data.columns = data.columns.str.strip().str.replace("'", "")
data.head()

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,1,48,80,1.020,1,0,?,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,2,7,50,1.020,4,0,?,normal,notpresent,notpresent,...,38,6000,?,no,no,no,good,no,no,ckd
2,3,62,80,1.010,2,3,normal,normal,notpresent,notpresent,...,31,7500,?,no,yes,no,poor,no,yes,ckd
3,4,48,70,1.005,4,0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,5,51,80,1.010,2,0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd


PREPROCESSING & ENCODING

In [ ]:
categorical_cols = ['rbc','pc','pcc','ba','htn','dm','cad','appet','pe','ane']

value_mapping = {
    'yes':1,'no':0,
    'present':1,'notpresent':0,
    'normal':1,'abnormal':0,
    'good':1,'poor':0,
    'ckd':1,'notckd':0,
    '?':np.nan
}

data['class'] = data['class'].replace(value_mapping)

for col in categorical_cols:
    data[col] = data[col].replace(value_mapping)

data = data.apply(pd.to_numeric, errors='coerce')
data = data.drop(columns=['id'])

X = data.drop(columns=['class']).values
y = data['class'].values

/tmp/ipython-input-3325476756.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['class'] = data['class'].replace(value_mapping)
/tmp/ipython-input-3325476756.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[col] = data[col].replace(value_mapping)


IMPUTASI MISSING VALUE

In [ ]:
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)
np.isnan(X).sum()
pd.DataFrame(X).head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,48.0,80.0,1.020,1.0,0.0,0.809717,1.0,0.0,0.0,121.000000,...,15.4,44.0,7800.0,5.200000,1.0,1.0,0.0,1.0,0.0,0.0
1,7.0,50.0,1.020,4.0,0.0,0.809717,1.0,0.0,0.0,147.864407,...,11.3,38.0,6000.0,4.702247,0.0,0.0,0.0,1.0,0.0,0.0
2,62.0,80.0,1.010,2.0,3.0,1.000000,1.0,0.0,0.0,423.000000,...,9.6,31.0,7500.0,4.702247,0.0,1.0,0.0,0.0,0.0,1.0
3,48.0,70.0,1.005,4.0,0.0,1.000000,0.0,1.0,0.0,117.000000,...,11.2,32.0,6700.0,3.900000,1.0,0.0,0.0,0.0,1.0,1.0
4,51.0,80.0,1.010,2.0,0.0,1.000000,1.0,0.0,0.0,106.000000,...,11.6,35.0,7300.0,4.600000,0.0,0.0,0.0,1.0,0.0,0.0


BALANCING DATA (SMOTE)

In [ ]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

NORMALISASI

In [ ]:
scaler = MinMaxScaler()
X_resampled = scaler.fit_transform(X_resampled)

NGUYEN–WIDROW INITIALIZATION

In [ ]:
def nguyen_widrow_init(n_input, n_hidden):
    beta = 0.7 * (n_hidden ** (1 / n_input))
    W = np.random.uniform(-0.5, 0.5, (n_input, n_hidden))

    for i in range(n_hidden):
        norm = np.linalg.norm(W[:, i])
        W[:, i] = beta * W[:, i] / norm

    b = np.random.uniform(-beta, beta, n_hidden)
    return W, b

# ==================================================
# ANN MODEL BUILDER (WITH NGUYEN–WIDROW)
# ==================================================
def build_ann_model():
    model = Sequential()
    model.add(Dense(10, input_dim=X_resampled.shape[1], activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))

    # Apply Nguyen–Widrow
    W, b = nguyen_widrow_init(
        n_input=X_resampled.shape[1],
        n_hidden=10
    )
    model.layers[0].set_weights([W, b])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model
    # ==================================================
    # EARLY STOPPING
    # ==================================================
    early_stop = EarlyStopping(
        monitor='loss',
        patience=10,
        restore_best_weights=True
    )


CROSS VALIDATION + TRAINING ANN

In [ ]:
# ==================================================
# STRATIFIED K-FOLD CROSS VALIDATION
# ==================================================
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

acc_scores = []
prec_scores = []
rec_scores = []
f1_scores = []

for fold, (train_idx, test_idx) in enumerate(skf.split(X_resampled, y_resampled), 1):
    print(f"\n===== Fold {fold} =====")

    model = build_ann_model()

    model.fit(
        X_resampled[train_idx],
        y_resampled[train_idx],
        epochs=1000,
        verbose=0,
        callbacks=[early_stop]
    )

    y_pred = (model.predict(X_resampled[test_idx]) > 0.5).astype(int).flatten()

    acc_scores.append(accuracy_score(y_resampled[test_idx], y_pred))
    prec_scores.append(precision_score(y_resampled[test_idx], y_pred))
    rec_scores.append(recall_score(y_resampled[test_idx], y_pred))
    f1_scores.append(f1_score(y_resampled[test_idx], y_pred))


===== Fold 1 =====


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

===== Fold 2 =====


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

===== Fold 3 =====


...

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step

===== Fold 4 =====


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

===== Fold 5 =====


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


HASIL AKHIR

In [ ]:
# ==================================================
# FINAL CV RESULTS
# ==================================================
print("\n===================================")
print("RATA-RATA AKURASI  :", np.mean(acc_scores))
print("STD AKURASI        :", np.std(acc_scores))
print("PRECISION (AVG)    :", np.mean(prec_scores))
print("RECALL (AVG)       :", np.mean(rec_scores))
print("F1-SCORE (AVG)     :", np.mean(f1_scores))
print("===================================")



RATA-RATA AKURASI  : 0.98989898989899
STD AKURASI        : 0.011065102171821528
PRECISION (AVG)    : 1.0
RECALL (AVG)       : 0.9799183673469388
F1-SCORE (AVG)     : 0.9897323752993856
